<a href="https://colab.research.google.com/github/waltz2u/bd/blob/master/Basic_Spark_RDD_MorningStar_Rating_and_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Spark RDD with  MorningStar Rating and Stock data
First mount Google Drive

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## Spark Installation on Google Colab

The following download OpenJDK Java, and Spark, unzip Spark to a directory on Google Drive, and install FindSpark, a library that makes it easy for Python to find Spark.

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [819 B]
Get:12 http://ppa.launchpad.net/marutter/c2d4u

Take a look at files and directories in the current directory (on Google Drive)

In [2]:
!ls

sample_data  spark-2.4.5-bin-hadoop2.7	spark-2.4.5-bin-hadoop2.7.tgz


Switch to Java 8 if it is Java 11 now. In the following enter the number that represents the installation of Java 8

In [4]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [5]:
!java -version 

openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)


Setting the Environment Variables (home directory) for Java and Spark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

This will start a local Spark session.

In [0]:
import findspark
findspark.init("/content/spark-2.4.5-bin-hadoop2.7")
from pyspark.sql import SparkSession
# Sets the Spark master URL to connect to, such as "local" to run locally, 
# "local[4]" to run locally with 4 cores, or "spark://master:7077" to run on a Spark standalone cluster.
spark = SparkSession.builder.master("local[*]").getOrCreate() #

## MorningStar Ratings

First read data in

In [12]:
lines = spark.sparkContext.textFile(r'/content/gdrive/My Drive/models/morningstar/morningstar.csv')
lines.take(3)

['ProFunds Short Precious Metals Inv,SPPIX,NA',
 'ProFunds Short Precious Metals Svc,SPPSX,NA',
 'ProFunds Biotechnology UltraSector Inv,BIPIX,4']

In [13]:
ratings = lines.map(lambda x: x.split(',')[2])
ratings.take(10)

['NA', 'NA', '4', '3', '5', '5', '4', '4', '4', '4']

In [14]:
result = ratings.countByValue()
print(result)

defaultdict(<class 'int'>, {'NA': 19, '4': 19, '3': 19, '5': 18, '1': 24})


In [15]:
import collections
sortedCounts = collections.OrderedDict(sorted(result.items()))
for key, value in sortedCounts.items():
    print ("%s %i" % (key, value))

1 24
3 19
4 19
5 18
NA 19


## Mean Price by Month

Given date and stock price, returns average price by month

In [16]:
# this function gets month, year and adjusted closed price
def parseLine(line):
    fields = line.split(',')
    datetimedata = fields[0]
    datetimearray = datetimedata.split('/')
    monthyear = datetimearray[0]+datetimearray[2]
    adjclose = float(fields[6])
    return (monthyear, adjclose)

lines = spark.sparkContext.textFile("/content/gdrive/My Drive/models/stocks/adjclose.csv")

# lines is an RDD, take some to check
lines.take(3)

['1/28/2015,49.799999,50.32,46.299999,46.459999,84839700,46.459999',
 '1/27/2015,49.139999,49.279999,47.66,47.990002,45777200,47.990002',
 '1/26/2015,49.57,49.790001,49.07,49.439999,18976400,49.439999']

In [17]:
# gets month, year and adjusted closed price
rdd = lines.map(parseLine)
rdd.take(3)

[('12015', 46.459999), ('12015', 47.990002), ('12015', 49.439999)]

In [18]:
# mapValues to gets a tuple of (adjusted price, 1) for each month
rdd.mapValues(lambda x: (x, 1)).take(3)

[('12015', (46.459999, 1)),
 ('12015', (47.990002, 1)),
 ('12015', (49.439999, 1))]

In [19]:
# then add tuples together using reduceByKey
priceByMonthYear = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
priceByMonthYear.take(3)

[('12015', (871.899997, 18)),
 ('112014', (952.880004, 19)),
 ('92014', (854.119997, 21))]

In [20]:
# then finally mapValues to calculate average price 
averagesByMonthYear = priceByMonthYear.mapValues(lambda x: x[0] / x[1])
results = averagesByMonthYear.collect()
for result in results:
    print (result)

('12015', 48.438888722222224)
('112014', 50.15157915789474)
('92014', 40.67238080952381)
('82014', 36.87619052380952)
('72014', 35.097272409090905)
('32014', 37.71476223809524)
('22014', 37.43578963157895)
('82013', 27.525454454545457)
('72013', 27.304545590909093)
('62013', 25.911499999999997)
('52013', 26.159545227272723)
('12013', 19.815714523809525)
('102012', 16.12428576190476)
('82012', 15.226086956521744)
('72012', 15.77476195238095)
('52012', 15.358181818181817)
('42012', 15.253499999999997)
('22012', 15.371500000000003)
('122011', 15.678571333333334)
('72011', 14.505)
('62011', 15.178181818181816)
('52011', 16.98523780952381)
('32011', 16.660869608695652)
('12011', 16.42199995)
('112010', 16.419523761904763)
('82010', 13.796363636363639)
('72010', 14.474761904761904)
('52010', 15.935499949999999)
('32010', 16.303913260869564)
('22010', 15.268947368421053)
('102009', 16.94318168181818)
('92009', 16.150952285714283)
('82009', 14.75095238095238)
('122014', 50.34045431818181)
('10